In [1]:
import numpy as np
import pandas as pd
import math
%pylab inline
from scipy.special import gamma

Populating the interactive namespace from numpy and matplotlib


## Graphical Models 

### Hierachical Models

* All covariates determine the response, hence use dashed lines, in Bayesian regression problem. 
* Everything that is not random denote as square box. 
* The full conditional distributions can be either be computed using Markov Blankets or the full joined distribution where all terms where the conditioned variable does not appear are thrown out. 

In [2]:
y_i = np.array([5, 1, 5, 14, 3, 19, 1, 1, 4, 22])
t_i = np.array([94.3, 15.7, 62.9, 125.8, 5.2, 31.4, 1.0, 1.0, 2.1, 10.5])
theta_t = y_i * t_i
data_table = pd.DataFrame(np.c_[y_i, t_i, theta_t], columns=["y_i", "t_i", "theta_t"])

In [3]:
data_table.T

,0,1,2,3,4,5,6,7,8,9
y_i,5.0,1.0,5.0,14.0,3.0,19.0,1,1,4.0,22.0
t_i,94.3,15.7,62.9,125.8,5.2,31.4,1,1,2.1,10.5
theta_t,471.5,15.7,314.5,1761.2,15.6,596.6,1,1,8.4,231.0


Assuming:

$$
Y_{i} | \theta_{i} \sim Poisson(\theta_{i} t_{i}) \\
$$

All probabilistic distributions: 

$$
Y_{i} | \theta_{i} \sim Poisson(\theta_{i} t_{i}) \\ \theta_{i} | \alpha \space \beta \sim Gamma(\alpha, \beta) \\
\alpha \sim Exponential(0.001) \\ \beta \sim Exponential(0.001)
$$


The joined Posterior is of the form: 

$$ 
p( \textbf{$\theta$}, \alpha, \beta \space | \space\textbf{y}) \propto\bigg\{ \prod^{N}_{i = 0}p(\theta_{i} \space | \space \alpha, \beta) \bigg \} \bigg \{ \prod^{N}_{i = 0} p(y_{i}\space | \theta_{i}) \bigg \} p(\alpha)p(\beta) \\
$$

$$ p( \textbf{$\theta$}, \alpha, \beta \space | \space\textbf{y}) \propto \prod e^{-\theta_{i}t_{i}}\frac{t_{i}\theta_{i}^{k}}{k!} \times \prod\frac{\beta^{\alpha}}{\Gamma\alpha}\theta_{i}^{\alpha-1}e^{-\beta \theta_{i}} \times 0.001e^{-0.001\alpha}\times 0.001e^{-0.001\beta} \\
$$

From here we can compute the full conditional posterior distirbutions

$$
p(\alpha \space|\space \beta, \theta_{i}) = \prod\frac{\beta^{\alpha}}{\Gamma\alpha}\theta_{i}^{\alpha-1}e^{-\beta \theta_{i}} \times 0.001e^{-0.001\alpha} \\ 
p(\beta \space|\space \alpha, \theta_{i}) = \prod\frac{\beta^{\alpha}}{\Gamma\alpha}\theta_{i}^{\alpha-1}e^{-\beta \theta_{i}} \times 0.001e^{-0.001\beta} \\
p(\alpha \space | \space \beta, \theta) = \bigg (\frac{\beta^{\alpha}}{\Gamma \alpha}\bigg )^{n} e^{(\alpha-1)\sum ln \theta_{i} - 0.001 \alpha}\\ $$

$$
p(\beta \space | \space \alpha, \theta) = \beta^{\alpha n } e^{-n\bar{\theta}\beta - 0.001\beta} \\
$$

Gibb Sampler: 

* Find all full conditional distributions.
* We set all variables to their initial value (sampling from the prior distribution).
* In this case we set:




$p(\theta \space|\space \alpha, \beta) = 0.001e^{-0.001\alpha}\times 0.001e^{-0.001\beta} \times \prod\frac{\beta^{\alpha}}{\Gamma\alpha}\theta_{i}^{\alpha-1}e^{-\beta \theta_{i}} $

In [4]:
initial_values = {
    "alpha": 1, 
    "beta": 2, 
}

In [67]:
sample_beta(0.00141971016708, 0.001, data_table)

-3.4165010000000002


0.029760564403886785

In [23]:
def sample_alpha(alpha, beta, data):
    s = sum(data["theta_t"])
    ln_s = sum(log(data["theta_t"]))
    n = len(data["theta_t"])
    gam = (beta**alpha / gamma(alpha))**n
    
    t1 = exp((alpha-1)*sum(ln_s)) - 0.999*log(alpha)
    t2 = exp(-beta*s - 0.001*alpha)
    print(t2*t1)

sample_alpha(0.1, 0.9, data_table)

0.0


In [2]:
gamma(0)

inf

In [31]:
X = np.random.normal(2.32, 0.4232, 100)

mu = 2
tau = 0.2

def sample_mu(mu, tau, x): 
    n = len(x)
    ta = (n*tau+10e-6)
    m = (n*tau/(10e-6+n*tau))*mean(x)
    
    t1 = sqrt(ta/(2*pi))
    t2 = exp(-(ta/2)*((tau-m)**2))
    return t1*t2
    
    
sample_mu(mu, tau, X)


3.2501578468880439e-20

In [45]:
4**(0.43232)

1.8208852395598014

In [27]:
data_table["theta_t"]

0     471.5
1      15.7
2     314.5
3    1761.2
4      15.6
5     596.6
6       1.0
7       1.0
8       8.4
9     231.0
Name: theta_t, dtype: float64

$$$$